使用資料增強技術搭配正則化(regularization)技術，例如，Dropout, Batch normalization

In [46]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Version A：原始資料（無增強）
raw_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225])
])

# Version B：訓練用資料增強
aug_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std =[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3,
                             scale=(0.02, 0.15),
                             ratio=(0.3, 3.3),
                             value=0)  # 或用 'random'
])

# 載入訓練／測試 Dataset
raw_train_full = datasets.ImageFolder(root="./data/train", transform=raw_transform)
raw_test     = datasets.ImageFolder(root="./data/test",  transform=raw_transform)

aug_train_full = datasets.ImageFolder(root="./data/train", transform=aug_transform)
#raw_transform 以評估增強後模型真實表現
aug_val_full  = datasets.ImageFolder(root="./data/train", transform=raw_transform)
aug_test      = datasets.ImageFolder(root="./data/test",  transform=raw_transform)

print(" 類別對應：", raw_train_full.classes)
print(" 訓練筆數（原始）：", len(raw_train_full))
print(" 測試筆數：", len(raw_test))

# 切分訓練 / 驗證集（8:2）
train_ratio = 0.8
raw_train_size = int(len(raw_train_full) * train_ratio)
raw_val_size   = len(raw_train_full) - raw_train_size
raw_train, raw_val = random_split(raw_train_full, [raw_train_size, raw_val_size])

aug_train_size = int(len(aug_train_full) * train_ratio)
aug_val_size   = len(aug_train_full) - aug_train_size
aug_train, _   = random_split(aug_train_full, [aug_train_size, len(aug_train_full)-aug_train_size])
_, aug_val     = random_split(aug_val_full, [aug_train_size, len(aug_val_full)-aug_train_size])

# DataLoader
batch_size = 64
raw_train_loader = DataLoader(raw_train, batch_size=batch_size, shuffle=True)
raw_val_loader   = DataLoader(raw_val,   batch_size=batch_size, shuffle=False)
raw_test_loader  = DataLoader(raw_test,  batch_size=batch_size, shuffle=False)

aug_train_loader = DataLoader(aug_train, batch_size=batch_size, shuffle=True)
aug_val_loader   = DataLoader(aug_val,   batch_size=batch_size, shuffle=False)
aug_test_loader  = DataLoader(aug_test,  batch_size=batch_size, shuffle=False)


 類別對應： ['men', 'women']
 訓練筆數（原始）： 220
 測試筆數： 80


In [47]:
import torch
import torch.nn as nn
import copy

def train_model(model, train_loader, val_loader, num_epochs, device, label="",):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-3) # lr = 2e-3 收斂太快
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)   
    
    best_acc = 0.0
    best_model_state_dict = copy.deepcopy(model.state_dict())

    print(f" 開始訓練：{label}")
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        # 訓練階段
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step() 
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        train_acc = 100 * correct / total

        # 驗證
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_acc = 100 * val_correct / val_total

        print(f"[{label}] Epoch {epoch+1}/{num_epochs}, Loss: {running_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_state_dict = copy.deepcopy(model.state_dict())

    # 訓練結束後加載最佳模型
    model.load_state_dict(best_model_state_dict)
    print(f"[{label}] 最佳驗證準確率：{best_acc:.2f}%")
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
def evaluate_model(model, test_loader, device, label=""):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"[{label}]  測試集準確率：{acc:.2f}%")
    return acc

Dropout

In [49]:
import torchvision.models as models
import torch.nn as nn

class GenderCNN_Dropout(nn.Module):
    def __init__(self):
        super().__init__()
        # 載入預訓練 EfficientNet‑B0
        self.backbone = models.efficientnet_b0(pretrained=True)
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in self.backbone.features[-5:].parameters():
            param.requires_grad = True

        # 替換分類頭，插入 Dropout
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(1280, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),

            nn.Linear(256, 2)
        )

    def forward(self, x):
        return self.backbone(x)


In [50]:
# 訓練參數
num_epochs = 20
model_raw = GenderCNN_Dropout().to(device)
print("\n" + "="*30 + " Training Raw Model " + "="*30)
model_raw = train_model(model_raw, raw_train_loader, raw_val_loader, num_epochs, device, label="raw")

model_aug = GenderCNN_Dropout().to(device)
print("\n" + "="*30 + " Training Augmented Model " + "="*30)
model_aug = train_model(model_aug, aug_train_loader, aug_val_loader, num_epochs, device, label="augmented")



============================== Training Raw Model ==============================
 開始訓練：raw
[raw] Epoch 1/20, Loss: 2.1641, Train Acc: 54.55%, Val Acc: 56.82%
[raw] Epoch 2/20, Loss: 1.6574, Train Acc: 69.89%, Val Acc: 65.91%
[raw] Epoch 3/20, Loss: 1.4730, Train Acc: 81.25%, Val Acc: 70.45%
[raw] Epoch 4/20, Loss: 1.2656, Train Acc: 86.36%, Val Acc: 75.00%
[raw] Epoch 5/20, Loss: 1.2484, Train Acc: 88.64%, Val Acc: 79.55%
[raw] Epoch 6/20, Loss: 1.1958, Train Acc: 87.50%, Val Acc: 79.55%
[raw] Epoch 7/20, Loss: 1.1234, Train Acc: 92.05%, Val Acc: 77.27%
[raw] Epoch 8/20, Loss: 1.1190, Train Acc: 90.34%, Val Acc: 79.55%
[raw] Epoch 9/20, Loss: 1.1328, Train Acc: 86.93%, Val Acc: 81.82%
[raw] Epoch 10/20, Loss: 1.0421, Train Acc: 93.18%, Val Acc: 84.09%
[raw] Epoch 11/20, Loss: 1.1211, Train Acc: 92.05%, Val Acc: 86.36%
[raw] Epoch 12/20, Loss: 1.0301, Train Acc: 92.61%, Val Acc: 88.64%
[raw] Epoch 13/20, Loss: 1.1072, Train Acc: 93.18%, Val Acc: 90.91%
[raw] Epoch 14/20, Loss: 0.9711, 

In [51]:
evaluate_model(model_raw, raw_test_loader, device, label="raw")
evaluate_model(model_aug, aug_test_loader, device, label="augmented")

[raw]  測試集準確率：76.25%
[augmented]  測試集準確率：81.25%


81.25

Batch normalization

In [52]:
import torchvision.models as models
import torch.nn as nn

class GenderCNN_BN(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.backbone = models.efficientnet_b0(pretrained=True)
    
        for param in self.backbone.parameters():
            param.requires_grad = False
        for param in self.backbone.features[-5:].parameters():
            param.requires_grad = True

        # 替換分類頭：1280 → 512 → 256 → 2
        # 在每層全連接之後都加 BN，再 ReLU
        self.backbone.classifier[1] = nn.Sequential(
            nn.Linear(1280, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            
            nn.Linear(256, 2)
        )

    def forward(self, x):
        return self.backbone(x)


In [53]:
# 訓練參數
num_epochs = 20  
model_raw = GenderCNN_Dropout().to(device)
print("\n" + "="*30 + " Training Raw Model " + "="*30)
model_raw = train_model(model_raw, raw_train_loader, raw_val_loader, num_epochs, device, label="raw")

model_aug = GenderCNN_Dropout().to(device)
print("\n" + "="*30 + " Training Augmented Model " + "="*30)


model_aug = train_model(model_aug, aug_train_loader, aug_val_loader, num_epochs, device, label="augmented")



============================== Training Raw Model ==============================
 開始訓練：raw
[raw] Epoch 1/20, Loss: 2.0960, Train Acc: 53.98%, Val Acc: 56.82%
[raw] Epoch 2/20, Loss: 1.7679, Train Acc: 69.89%, Val Acc: 61.36%
[raw] Epoch 3/20, Loss: 1.5706, Train Acc: 77.84%, Val Acc: 68.18%
[raw] Epoch 4/20, Loss: 1.3982, Train Acc: 81.25%, Val Acc: 77.27%
[raw] Epoch 5/20, Loss: 1.3132, Train Acc: 83.52%, Val Acc: 86.36%
[raw] Epoch 6/20, Loss: 1.2848, Train Acc: 88.07%, Val Acc: 88.64%
[raw] Epoch 7/20, Loss: 1.1929, Train Acc: 89.20%, Val Acc: 88.64%
[raw] Epoch 8/20, Loss: 1.1650, Train Acc: 89.20%, Val Acc: 84.09%
[raw] Epoch 9/20, Loss: 1.0692, Train Acc: 90.91%, Val Acc: 84.09%
[raw] Epoch 10/20, Loss: 1.1541, Train Acc: 89.77%, Val Acc: 86.36%
[raw] Epoch 11/20, Loss: 1.1396, Train Acc: 89.20%, Val Acc: 88.64%
[raw] Epoch 12/20, Loss: 1.0564, Train Acc: 94.89%, Val Acc: 86.36%
[raw] Epoch 13/20, Loss: 1.1887, Train Acc: 87.50%, Val Acc: 86.36%
[raw] Epoch 14/20, Loss: 1.1437, 

In [54]:
evaluate_model(model_raw, raw_test_loader, device, label="raw")
evaluate_model(model_aug, aug_test_loader, device, label="augmented")

[raw]  測試集準確率：68.75%
[augmented]  測試集準確率：82.50%


82.5